# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9c4c605be0>
├── 'a' --> tensor([[ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632]])
└── 'x' --> <FastTreeValue 0x7f9c70619ca0>
    └── 'c' --> tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                        [ 0.4235,  0.9042,  0.7096,  0.1917],
                        [ 0.0754, -0.5071, -0.8235, -0.5396]])

In [4]:
t.a

tensor([[ 0.0728, -0.5750, -0.7938],
        [ 0.2080, -0.7280,  0.2632]])

In [5]:
%timeit t.a

68.3 ns ± 0.0288 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9c4c605be0>
├── 'a' --> tensor([[ 1.8406,  0.6986,  0.6411],
│                   [ 1.1580, -0.1093, -0.8276]])
└── 'x' --> <FastTreeValue 0x7f9c70619ca0>
    └── 'c' --> tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                        [ 0.4235,  0.9042,  0.7096,  0.1917],
                        [ 0.0754, -0.5071, -0.8235, -0.5396]])

In [7]:
%timeit t.a = new_value

66.7 ns ± 0.051 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632]]),
    x: Batch(
           c: tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                      [ 0.4235,  0.9042,  0.7096,  0.1917],
                      [ 0.0754, -0.5071, -0.8235, -0.5396]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0728, -0.5750, -0.7938],
        [ 0.2080, -0.7280,  0.2632]])

In [11]:
%timeit b.a

53.6 ns ± 0.0261 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6381, -0.2176,  0.1853],
               [ 1.3131,  0.6267, -0.3765]]),
    x: Batch(
           c: tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                      [ 0.4235,  0.9042,  0.7096,  0.1917],
                      [ 0.0754, -0.5071, -0.8235, -0.5396]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.0899 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

862 ns ± 3.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9bc32c5b20>
├── 'a' --> tensor([[[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]],
│           
│                   [[ 0.0728, -0.5750, -0.7938],
│                    [ 0.2080, -0.7280,  0.2632]]])
└── 'x' --> <FastTreeValue 0x7f9bc32c5340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9bc32c51c0>
├── 'a' --> tensor([[ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632],
│                   [ 0.0728, -0.5750, -0.7938],
│                   [ 0.2080, -0.7280,  0.2632]])
└── 'x' --> <FastTreeValue 0x7f9bc32c57f0>
    └── 'c' --> tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                        [ 0.4235,  0.9042,  0.7096,  0.1917],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 56 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56 µs ± 53.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]],
       
               [[ 0.0728, -0.5750, -0.7938],
                [ 0.2080, -0.7280,  0.2632]]]),
    x: Batch(
           c: tensor([[[ 0.0846,  0.1408,  0.4734, -0.5364],
                       [ 0.4235,  0.9042,  0.7096,  0.1917],
                       [ 0.0754, -0.5071, -0.8235, -0.5396]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632],
               [ 0.0728, -0.5750, -0.7938],
               [ 0.2080, -0.7280,  0.2632]]),
    x: Batch(
           c: tensor([[ 0.0846,  0.1408,  0.4734, -0.5364],
                      [ 0.4235,  0.9042,  0.7096,  0.1917],
                      [ 0.0754, -0.5071, -0.8235, -0.5396],
                      [ 0.0846,  0.1408,  0.4734, -0.5364],
                      [ 0.4235,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 443 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

300 µs ± 590 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
